# Finding optima in 1 dimension



We will animate the steps of the minmax-finding algorithms following the nice instructions [here](https://stackoverflow.com/questions/9401658/how-to-animate-a-scatter-plot). 

In [ ]:
%matplotlib ipympl
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.widgets import Button, Slider

import scipy.optimize
from scipy.optimize import elementwise

plt.rcParams.update({
    "axes.labelsize": "large", 
    "xtick.labelsize": "large", 
    "ytick.labelsize": "large"
})


## Define the function

Also draw the function within a range "close" to our root. 

In [ ]:
def f(x) : 
    return -0.5 * (x**2.0) + 0.25 * (x**4.0)

In [ ]:
x1 = -2.5
x2 = 2.5
y1 = -0.5
y2 = 8.

xvals = np.linspace(x1,x2,1000)
yvals = f(xvals)

fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(xvals,yvals)
ax.set_xlabel("x")
ax.set_xlim(x1, x2)
ax.set_ylim(y1, y2)
ax.set_ylabel(r"$-\frac{1}{2}x^2 + \frac{1}{4}x^4$")
ax.grid()
#ax.set_yscale("log")

## Make a class to animate the steps of the algorithm

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML
rc('animation', html='html5')

In [ ]:
class AnimatedScatter(object):
    """An animated scatter plot using matplotlib.animations.FuncAnimation."""
    def __init__(self, iterations, fig, ax, color=1):
        self.iterations=iterations
        self.numpoints = iterations.shape[0]
        self.fig=fig
        self.ax=ax
        self.color=color
        self.i = 0        
        self.stream = self.data_stream()
        self.ani = animation.FuncAnimation(self.fig, self.update, interval=500,
                                           frames=self.iterations.T.shape[1],
                                           init_func=self.setup_plot, blit=False)

    def setup_plot(self):
        """Initial drawing of the scatter plot."""
        x,y = next(self.stream)
        self.scat = self.ax.scatter(x, y, c=np.full_like(x,self.color), vmin=0, vmax=10,
                                    cmap="jet", edgecolor="k")
        #self.ax.axis([self.x1,self.x2,self.y1,self.y2])
        
        return self.scat,

    def data_stream(self):
        while True:
            yield np.array( [self.iterations[self.i:self.i+1,0],self.iterations[self.i:self.i+1,1]] )

    def update(self, i):
        """Update the scatter plot."""
        self.i = i
        data = next(self.stream)        

        # Set x and y data...
        self.scat.set_offsets(data[0:2].T)

        # We need to return the updated artist for FuncAnimation to draw..
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,

## Find the optima

Also define a "callback" function to add the steps of the algorithm to a path. 

In [ ]:
def make_minimize_cb(path=[]):    
    def minimize_cb(xk):
        # note that we make a deep copy of xk
        path.append(np.copy(xk))
    return minimize_cb

### First try the golden section search

Note: `scipy` has a bug in the printing here, so we won't animate it. 

In [ ]:
tol1 = 1e-6
res1 = scipy.optimize.minimize_scalar(fun=f, bracket=(0,0.5,2), method="golden", tol=tol1, options={"disp": 3}) 
print(res1)

In [ ]:
# Now Brent's method
res = scipy.optimize.minimize_scalar(fun=f, bracket=(0,0.5,2), method="brent", tol=1.e-6, options={"disp": 3}) 
print(res)

### Next try the BFGS search

We will use the default

In [ ]:
a2 = 3
tol2 = 1e-6
pathx2 = [np.array([a2])]
res2 = scipy.optimize.minimize(fun=f, x0=a2, tol=tol2, method="BFGS", callback=make_minimize_cb(pathx2))
print(pathx2)
xmin2 = res2.x
fxmin2 = res2.fun
print("Original pathx2:")
print(pathx2)
pathx2 = np.array(pathx2)
pathy2 = f(pathx2)
print(xmin2, fxmin2)
path2 = np.concatenate( [pathx2, pathy2], axis=1)
print(path2)

In [ ]:
anim2 = AnimatedScatter(iterations=path2, fig=fig, ax=ax)
HTML(anim2.ani.to_html5_video())

In [ ]:
# Try newer Chandrupatla
def make_minimize_cb2(xpath=[], ypath=[]):    
    def minimize_cb(xk):
        # note that we make a deep copy of xk
        xpath.append(np.copy(xk["x"]).item())
        ypath.append(np.copy(xk["f_x"]).item())
    return minimize_cb
    
#a3 = 0.5
tol3 = 1e-6
pathx3 = []
pathy3 = []
res3 = elementwise.find_minimum(f, (0,0.5,2), tolerances={"xatol": tol3}, callback=make_minimize_cb2(pathx3, pathy3)) 

pathx3 = np.array(pathx3)
pathy3 = np.array(pathy3)
path3 = np.column_stack([pathx3, pathy3])
print(path3)

In [ ]:
# Just for fun, here's a slider bar plot instead of an animation
class SliderPlot:
    def __init__(self, xx, yy):
        self._x = xx
        self._y = yy
        
        self._fig, self._ax = plt.subplots(figsize=(8, 6))
        self._fig.subplots_adjust(bottom=0.25)
        
        self._ax.plot(xvals, yvals)
        self._ax.set_xlabel("x")
        self._ax.set_ylabel(r"$-\frac{1}{2}x^2 + \frac{1}{4}x^4$")
        self._ax.set_xlim(0.25, 1.75)
        self._ax.set_ylim(-0.3, 0.0)
        self._ax.grid()

        # Plot the first point only
        self._scatter_points = self._ax.scatter(self._x[0], self._y[0])

        nsteps = len(self._x)
        self._ax_slider = self._fig.add_axes([0.25, 0.1, 0.65, 0.03])
        self._step_slider = Slider(
            self._ax_slider, "Step", 0, nsteps,
            valinit=0, valstep=np.arange(nsteps),
            color="green"
        )
        self._step_slider.on_changed(self.update)


    def update(self, val):
        #i = step_slider.val
        print(val)
        self._scatter_points.set_offsets([self._x[val], self._y[val]])
        self._fig.canvas.draw_idle()

#step_slider.on_changed(update)
#plt.show()

print(pathx3)
print(pathy3)
s = SliderPlot(pathx3, pathy3)